In [ ]:
!pip install torch
!pip install classy_vision
!pip install tensorboard
%load_ext tensorboard
!pip install psutil
!pip install segmentation-models-pytorch==0.1.3

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from shutil import copyfile
copyfile('drive/MyDrive/Colab Notebooks/SIGNATEHiroshimaLemon/data/archive.hdf5','archive.hdf5')

In [ ]:
import os
import re
import h5py
import numpy as np
import torch.utils.data
from torch.utils.data import random_split
from classy_vision.dataset import ClassyDataset
from classy_vision.tasks import ClassificationTask
import torchvision.transforms as transforms
from classy_vision.dataset.transforms import build_transforms, ApplyTransformToKey
from classy_vision.dataset.transforms import LightingTransform
from PIL import Image
from math import ceil
import psutil
cpu_count = psutil.cpu_count()

import logging
import os
logger = logging.getLogger()
logger.setLevel(os.environ.get("LOGLEVEL", "INFO"))


class HiroshimaLemon(torch.utils.data.Dataset):
    """HiroshimaLemon dataset."""

    def __init__(self, data='archive.hdf5', split='train', in_memory=True):
        assert os.path.exists(data), "Data path '{}' not found".format(data)
        splits = ["train", "test"]
        assert split in splits, "Split '{}' not supported".format(split)
        logger.info("Constructing HiroshimaLemon {}...".format(split))
        self._data, self._split = data, split
        self._size = 0
        assert in_memory, "Only in_memory implemented"
        self._in_memory = in_memory
        self._construct_imdb()

    def _construct_imdb(self):
        """Constructs the imdb."""

        with h5py.File(self._data,'r') as hf:

            if self._in_memory:
                self._imdb = {}
            if self._split == 'train':
                self._imdb_lookup = {}
                i = 0
                train = hf['train']
                self._class_ids = sorted(f for f in train.keys() if re.match(r"^[0-9]+$", f))
                self._class_id_cont_id = {v: i for i, v in enumerate(self._class_ids)}
                for class_id in self._class_ids:
                    cont_id = self._class_id_cont_id[class_id]
                    if self._in_memory:
                        self._imdb[cont_id] = train[class_id][()]
                    size = train[class_id].shape[0]
                    self._imdb_lookup.update(dict(zip(list(range(i,i+size)),list(zip(list(range(0,size)),[cont_id]*size)))))
                    i += size
                self._size = i
                logger.info("Number of images: {}".format(i))
                logger.info("Number of classes: {}".format(len(self._class_ids)))
            else:
                if self._in_memory:
                    self._imdb = hf['test'][()]
                    size = self._imdb.shape[0]
                    logger.info("Number of images: {}".format(size))
                    self._size = size

    def __getitem__(self, index):
        assert index >= 0 and index < self._size, \
            "Provided index {} must be in range [0, {}).".format(index, self._size)
        if self._split == 'train':
          i, cont_id = self._imdb_lookup[index]
          return {'input':Image.fromarray(self._imdb[cont_id][i,:,:,:], 'RGB'),'target':cont_id}
        else:
          return Image.fromarray(self._imdb[index,:,:,:], 'RGB')

    def __len__(self):
        return self._size

class HiroshimaLemonClassy(ClassyDataset):
    def __init__(self, dataset, batchsize_per_replica, shuffle, transform, num_samples):
        super().__init__(dataset, batchsize_per_replica, shuffle, transform, num_samples)

dataset = HiroshimaLemon()

# Transformation pipeline
image_transform_train = transforms.Compose([
    transforms.RandomRotation(180, resample=False, expand=False),
    transforms.RandomPerspective(distortion_scale=0.05, p=0.5, interpolation=2, fill=0),
    transforms.RandomResizedCrop(240, scale=(2.66-0.2,2.66+0.2), ratio=(0.9, 1.1), interpolation=2),
    transforms.ToTensor(),
    LightingTransform(alphastd=0.1),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

transform_train = ApplyTransformToKey(
    transform=image_transform_train,
    key="input",
)

image_transform_test = transforms.Compose([
    transforms.Resize(240),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

valid = False

if valid:

  transform_test = ApplyTransformToKey(
      transform=image_transform_test,
      key="input",
  )

  train_test_split = 0.1
  dataset_train, dataset_test = random_split(dataset, [len(dataset) - ceil(len(dataset) * train_test_split), ceil(len(dataset) * train_test_split)])

  dataset_train_classy = HiroshimaLemonClassy(
      dataset=dataset_train,
      batchsize_per_replica=8, 
      shuffle=True, 
      transform=transform_train,
      num_samples=None
  )

  dataset_test_classy = HiroshimaLemonClassy(
      dataset=dataset_test,
      batchsize_per_replica=8,
      shuffle=True, 
      transform=transform_test, 
      num_samples=None
  )
  dataset_test_classy.set_num_workers(cpu_count)

else:

  dataset_test = HiroshimaLemon(split='test')

  dataset_train_classy = HiroshimaLemonClassy(
      dataset=dataset,
      batchsize_per_replica=8, 
      shuffle=True, 
      transform=transform_train,
      num_samples=None
  )

dataset_train_classy.set_num_workers(cpu_count)

In [ ]:
import matplotlib.pyplot as plt
from random import randint

classes = set()
fig, axs = plt.subplots(nrows=1, ncols=4, figsize=(16, 5))
axs = axs.flatten()
while True:
  example = dataset[randint(0,len(dataset)-1)]
  target = example['target']
  if target in classes:
    pass
  else:
    classes.add(target)
    axs[target].imshow(example['input'])
    axs[target].title.set_text(target)
    axs[target].set_axis_off()
  if len(classes)==4:
    break
plt.tight_layout()
fig.suptitle('Examples of lemons in the dataset: 0-high, 3-low quality')
fig.subplots_adjust(top=0.92)
fig.savefig('example_lemons.png')

In [ ]:
%tensorboard --logdir .

In [ ]:
from classy_vision.tasks import ClassificationTask
from classy_vision.optim import SGD, Adam
from classy_vision.optim.param_scheduler import LinearParamScheduler
from classy_vision.trainer import LocalTrainer
from classy_vision.models import build_model
from classy_vision.losses import build_loss
from classy_vision.meters import AccuracyMeter
from classy_vision.hooks import ProgressBarHook, LossLrMeterLoggingHook, CheckpointHook, TensorboardPlotHook
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
from pathlib import Path
import segmentation_models_pytorch as smp
from collections import defaultdict
from datetime import datetime

now = datetime.now().strftime("%Y%m%d%H%M%S")

classification_model = build_model({"name": "efficientnet",
                     "model_name":"B1",
                     "num_classes":4,
                     "bn_momentum":0.010,
                     "bn_epsilon":1e-3,
                     "width_divisor":8,
                     "min_width":None,
                     "drop_connect_rate":0.2,
                     "use_se":False})


model_type = 'UnetPlusPlus'
#model_type = 'Linknet'
#model_type = 'FPN'
#model_type = 'PSPNet'

extra = defaultdict(dict)
extra['UnetPlusPlus'] = dict(decoder_attention_type=None,
                             decoder_use_batchnorm=True,
                             decoder_channels=(128, 64, 32, 16),
                             encoder_depth=4)

segmentation_model = getattr(smp, model_type)(
    encoder_name="efficientnet-b1",
    encoder_weights=None,
    in_channels=3,
    classes=1,
    activation='sigmoid',
    **extra[model_type]
)


#filename = 'unetplusplus_efficientnet_b1_20210223111032.pt'
#filename = 'pspnet_efficientnet_b1_20210228042756_15.pt' # So bad!
filename = 'unetplusplus_efficientnet_b1_20210228032202_8.pt'

copyfile(f'drive/MyDrive/Colab Notebooks/SIGNATEHiroshimaLemon/models/{filename}',filename)
segmentation_model.load_state_dict(torch.load(filename))

class Model(torch.nn.Module):
    def __init__(self, segmentation_model, classification_model):
        super(Model, self).__init__()
        self.segmentation_model = segmentation_model
        for param in self.segmentation_model.parameters():
            param.requires_grad = False
        self.segmentation_model.eval()
        self.classification_model = classification_model
        self.training = True

    def forward(self, x):
        mask = self.segmentation_model(x)
        return self.classification_model(mask*x)

    def train(self, mode=True):
        r"""Sets the module in training mode."""      
        self.training = mode
        for module in self.classification_model.children():
            module.train(mode)
        return self

    def eval(self):
        r"""Sets the module in evaluation mode."""
        return self.classification_model.train(False)

model = Model(segmentation_model,classification_model)

loss = build_loss({"name": "CrossEntropyLoss",
                   "reduction": "sum"})

#optimizer = SGD(momentum=0.9, weight_decay=1e-4, nesterov=True)
optimizer = Adam(weight_decay=1e-4)

suffix = datetime.now().isoformat()
base_folder = f"{os.path.dirname(os.path.abspath('__file__'))}/output_{suffix}"
try:
    os.makedirs(base_folder)
except OSError:
  pass
logger.info(f"Logging outputs to {base_folder}")

num_epoch = 400

if valid:
  task = ClassificationTask() \
          .set_model(model) \
          .set_dataset(dataset_train_classy, "train") \
          .set_dataset(dataset_test_classy, "test") \
          .set_loss(loss) \
          .set_optimizer(optimizer) \
          .set_optimizer_schedulers({"lr": LinearParamScheduler(start_value=0.01, end_value=0.0005)}) \
          .set_num_epochs(num_epoch) \
          .set_meters([AccuracyMeter([1,2])]) \
          .set_dataloader_mp_context('fork') \
          .set_hooks([ProgressBarHook(),LossLrMeterLoggingHook(),TensorboardPlotHook(SummaryWriter(log_dir=Path(base_folder) / "tensorboard"))])

else:
  task = ClassificationTask() \
          .set_model(model) \
          .set_dataset(dataset_train_classy, "train") \
          .set_loss(loss) \
          .set_optimizer(optimizer) \
          .set_optimizer_schedulers({"lr": LinearParamScheduler(start_value=0.01, end_value=0.0005)}) \
          .set_num_epochs(num_epoch) \
          .set_meters([AccuracyMeter([1,2])]) \
          .set_dataloader_mp_context('fork') \
          .set_hooks([ProgressBarHook(),LossLrMeterLoggingHook(),TensorboardPlotHook(SummaryWriter(log_dir=Path(base_folder) / "tensorboard"))])

trainer = LocalTrainer()
trainer.train(task)

In [ ]:
save_type = 1

model.eval()

if save_type==0:
  with torch.no_grad():
    script = torch.jit.trace(model, torch.randn(1, 3, 240, 240, dtype=torch.float).cuda())

    if valid:
      #torch.save(model.state_dict(), f'lemon_efficientnet_b1_valid_{now}_{num_epoch}.pt')
      torch.jit.save(script, f'lemon_efficientnet_b1_valid_{now}_{num_epoch}.pt')
      copyfile(f'lemon_efficientnet_b1_valid_{now}_{num_epoch}.pt',f'drive/MyDrive/Colab Notebooks/SIGNATEHiroshimaLemon/models/lemon_efficientnet_b1_valid_{now}_{num_epoch}.pt')
    else:
      #torch.save(model.state_dict(), f'lemon_efficientnet_b1_{now}_{num_epoch}.pt')
      torch.jit.save(script, f'lemon_efficientnet_b1_{now}_{num_epoch}.pt')
      copyfile(f'lemon_efficientnet_b1_{now}_{num_epoch}.pt',f'drive/MyDrive/Colab Notebooks/SIGNATEHiroshimaLemon/models/lemon_efficientnet_b1_{now}_{num_epoch}.pt')
elif save_type==1:
  torch.save(model.state_dict(), f'lemon_efficientnet_b1_{now}_{num_epoch}_type1.pt')
  copyfile(f'lemon_efficientnet_b1_{now}_{num_epoch}_type1.pt',f'drive/MyDrive/Colab Notebooks/SIGNATEHiroshimaLemon/models/lemon_efficientnet_b1_{now}_{num_epoch}_type1.pt')
elif save_type==2:
  torch.save(model, f'lemon_efficientnet_b1_{now}_{num_epoch}_type2.pt')
  copyfile(f'lemon_efficientnet_b1_{now}_{num_epoch}_type2.pt',f'drive/MyDrive/Colab Notebooks/SIGNATEHiroshimaLemon/models/lemon_efficientnet_b1_{now}_{num_epoch}_type2.pt')

In [ ]:
from torch import no_grad

total_test = len(dataset_test)

if valid:

  from sklearn.metrics._plot.confusion_matrix import ConfusionMatrixDisplay

  result = [[0 for _ in range(4)] for _ in range(4)]

  model.eval()
  with no_grad():
      for i in range(total_test):
        test = dataset_test_classy[i]
        predict = model(test['input'][None, :, :, :].cuda())
        predict_softmax = torch.log_softmax(predict, dim = 1)
        _, predict_class = torch.max(predict_softmax, dim = 1)
        result[test['target']][predict_class.item()] += 1

  result = np.array(result)
  result = result.astype('float') / result.sum(axis=1)[:, np.newaxis]

  fig, ax = plt.subplots(1,1, figsize=(10, 10))
  ConfusionMatrixDisplay(confusion_matrix=result, display_labels=[i for i in range(4)]).plot(include_values=True, ax=ax)
  fig.savefig('confusion.png')
  copyfile('confusion.png',f'drive/MyDrive/Colab Notebooks/SIGNATEHiroshimaLemon/models/confusion_lemon_efficientnet_b1_{now}_{num_epoch}.png',)

else:

  import pandas as pd

  result = []

  model.eval()
  with no_grad():
      for i in range(total_test):
        test = image_transform_test(dataset_test[i])
        predict = model(test[None, :, :, :].cuda())
        predict_softmax = torch.log_softmax(predict, dim = 1)
        _, predict_class = torch.max(predict_softmax, dim = 1)
        result.append([f'test_{i:04}.jpg',predict_class.item()])

  df = pd.DataFrame(result,columns=['id','class_num'])
  df.to_csv(f'lemon_efficientnet_b1_{now}_{num_epoch}.csv', index=False)
  copyfile(f'lemon_efficientnet_b1_{now}_{num_epoch}.csv',f'drive/MyDrive/Colab Notebooks/SIGNATEHiroshimaLemon/models/lemon_efficientnet_b1_{now}_{num_epoch}.csv')